In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from rdkit import Chem

In [12]:
# IMS data from Predicting differential ion mobility behaviour in silico using machine learning 
# --authors: Christian Ieritano, J. Larry Campbella, and W. Scott Hopkins
file_path = '/home/cmdunham/ChemicalDataGeneration/data/predicting_ion_mobility_iertiano_et_al_data.csv'
data = pd.read_csv(file_path)

# Importing the MoNA database of mass spec data, just for use as database to go from compound names to smiles and inchikeys. I haven't found a better way to do this efficiently.

file_path = '/home/cmdunham/ChemicalDataGeneration/data/MoNA_data/MoNA-export-GC-MS_Spectra.sdf'
mona_database = Chem.SDMolSupplier(file_path)

In [6]:
data.head()

,Compound,m/z,SV 1500,SV 2000,SV 2500,SV 3000,SV 3250,SV 3500,SV 3750,SV 4000,Boltzmann-weighted CCS
0,5-Cl-MQOH,194.04,0.1,0.1,0.3,0.5,0.91,1.6,2.57,3.8,138.66
1,5-CN-MQOH,185.07,0.1,0.1,-0.2,-0.3,-0.14,0.2,0.77,1.6,132.21
2,5-F-MQOH,178.07,-0.2,-0.4,-0.5,-0.3,0.08,0.7,1.60,2.8,127.47
3,5-Me-MQOH,174.09,-0.2,-0.2,0.0,0.8,1.47,2.4,3.65,5.3,129.45
4,5-NO2-MQOH,205.06,0.2,-0.1,0.0,-0.4,-0.34,0.0,0.59,1.4,137.66


In [7]:
compounds = list(data['Compound'])